In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Prose/Mathieu/Benter-Project

/home/mathieu/Prose/Mathieu/Benter-Project


In [2]:
from models.horse import Horse
from models.runner import Runner
from models.race import Race
from database.setup import create_sqlalchemy_session
import pandas as pd
import tqdm
import random
import re
from tabulate import tabulate
import os
import json
import datetime as dt
from typing import List
import sqlalchemy as sa
from collections import Counter
from  sqlalchemy.sql.expression import func
import numpy as np

from utils.music import parse_music, parse_unibet_music, MusicRank
from utils.logger import setup_logger
from constants import UnibetHorseSex

BATCH_SIZE = int(1e2)
UNIBET_DATA_PATH = "./data/Unibet"
with create_sqlalchemy_session() as db_session:
    horse=db_session.query(Horse).first()
    
logger = setup_logger(__name__)

In [18]:
def get_feature_for_runners(runner_ids: List[int], db_session)->dict:
    
    horse_query = f"""
    select
	r.id,
	races.date,
	r.music,
	runners_with_history.n_horse_previous_races,
	runners_with_history.n_horse_previous_positions,
	runners_with_history.average_horse_position,
	runners_with_history.average_horse_top_1,
	runners_with_history.average_horse_top_3
from
	runners r
join races on
	r.race_id = races.id
left join(
	select
		r.id,
		count(previous_horse_runners.id) as n_horse_previous_races,
		count(cast( previous_horse_runners."position" as integer)) as n_horse_previous_positions,
		AVG(cast( previous_horse_runners."position" as integer)) as average_horse_position,
		AVG(cast(previous_horse_runners.top_1 as integer)) as average_horse_top_1,
		AVG(cast(previous_horse_runners.top_3 as integer)) as average_horse_top_3
	from
		runners r
	join races on
		races.id = r.race_id
	join (
		select
			r1.*,
			cast(r1."position" as integer)= 1 as top_1,
			cast(r1."position" as integer)<= 3 as top_3
		from
			runners r1) previous_horse_runners on
		previous_horse_runners.horse_id = r.horse_id
	join races previous_horse_races on
		previous_horse_races.id = previous_horse_runners.race_id
	where
		previous_horse_races.date < races.date
		and r.id in ({','.join(str(runner_id) for runner_id in runner_ids)})
	group by
		r.id,
		races.date) runners_with_history on
	runners_with_history.id = r.id
where
	r.id in ({','.join(str(runner_id) for runner_id in runner_ids)})
    """
    
    df_features =  pd.DataFrame(db_session.execute(horse_query).fetchall(), 
                        columns=['runner_id','race_date', 'music', 'n_horse_previous_races', 'n_horse_previous_positions',
                                 'average_horse_position','average_horse_top_1', 'average_horse_top_3'])
    df_features.set_index('runner_id', inplace=True)
    
    
    
    for entity_name in ['jockey', 'trainer', 'owner']:
        query = f"""
select
	r.id,
    r.{entity_name}_id,
	runners_with_history.n_{entity_name}_previous_races,
	runners_with_history.n_{entity_name}_previous_positions,
	runners_with_history.average_{entity_name}_position,
	runners_with_history.average_{entity_name}_top_1,
	runners_with_history.average_{entity_name}_top_3
from
	runners r
join races on
	r.race_id = races.id
left join(
        select
            r.id,
            r.{entity_name}_id,
            count(previous_{entity_name}_runners.id) as n_{entity_name}_previous_races,
            count(cast( previous_{entity_name}_runners."position" as integer)) as n_{entity_name}_previous_positions,
            AVG(cast( previous_{entity_name}_runners."position" as integer)) as average_{entity_name}_position,
            AVG(cast(previous_{entity_name}_runners.top_1 as integer)) as average_{entity_name}_top_1,
            AVG(cast(previous_{entity_name}_runners.top_3 as integer)) as average_{entity_name}_top_3
        from
            runners r
        join races on
            races.id = r.race_id
        join (select r1.*, cast(r1."position" as integer)=1  as top_1, cast(r1."position" as integer)<=3  as top_3 from runners r1) previous_{entity_name}_runners on
            previous_{entity_name}_runners.{entity_name}_id = r.{entity_name}_id 
        join races previous_{entity_name}_races on
            previous_{entity_name}_races.id = previous_{entity_name}_runners.race_id
        where
            previous_{entity_name}_races.date < races.date
            and r.id in ({','.join(str(runner_id) for runner_id in runner_ids)})
        group by r.id) runners_with_history on
	runners_with_history.id = r.id
where
	r.id in ({','.join(str(runner_id) for runner_id in runner_ids)})"""
        df_feature_sub = pd.DataFrame(db_session.execute(query).fetchall(), 
                                      columns=['runner_id', f'{entity_name}_id', f'n_{entity_name}_previous_races', 
                                               f'n_{entity_name}_previous_positions', f'average_{entity_name}_position',
                                              f'average_{entity_name}_top_1', f'average_{entity_name}_top_3'])
        df_feature_sub.set_index('runner_id', inplace=True)
        df_features=df_features.join(df_feature_sub, on='runner_id')
    df_features.fillna(value=np.nan, inplace=True)
    return df_features
                                  
       

In [19]:
with create_sqlalchemy_session() as db_session:
    runner_ids = [r[0] for r in db_session.query(Runner.id).all()]

In [20]:
runner_ids[:10]

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [21]:
with create_sqlalchemy_session() as db_session:
    df_features=get_feature_for_runners(runner_ids=runner_ids[:100], db_session=db_session)
   

In [22]:
df_features.shape

(100, 25)

def fusion_horse_feature(row):
    parsed_music=parse_unibet_music(row.race_date.year, row.music)

    mean_place, win_ratio =np.nan, np.nan
    if parsed_music and len(parsed_music.events) > row.n_horse_previous_races:
        mean_place =np.mean([event.rank.value for event in parsed_music.events if isinstance(event.rank.value, int) and event.rank != MusicRank.TENTH_AND_BELOW])
        win_ratio=np.mean([event.rank == MusicRank.FIRST for event in parsed_music.events])

    elif row.n_horse_previous_races:
        mean_place = row.average_horse_position
        win_ratio=row.average_horse_top_1

    return pd.Series({'mean_horse_place': mean_place, 'average_horse_top_1':win_ratio})

In [27]:
df_features=pd.merge(df_features, df_features.apply(fusion_horse_feature, axis=1),left_index=True, right_index=True)

AttributeError: 'Series' object has no attribute 'average_horse_top_1'

In [28]:
for index, row in df_features.iterrows():
    print(index, fusion_horse_feature(row))

AttributeError: 'Series' object has no attribute 'average_horse_top_1'

In [29]:
row

race_date                                      2005-06-18
music                           6o 1o (04) 4o 2o 1o 1o 3o
n_horse_previous_races                                NaN
n_horse_previous_positions                            NaN
average_horse_position                                NaN
average_horse_top_1_x                                 NaN
average_horse_top_3                                   NaN
jockey_id                                               3
n_jockey_previous_races                               NaN
n_jockey_previous_positions                           NaN
average_jockey_position                               NaN
average_jockey_top_1                                  NaN
average_jockey_top_3                                  NaN
trainer_id                                              2
n_trainer_previous_races                              NaN
n_trainer_previous_positions                          NaN
average_trainer_position                              NaN
average_traine

In [25]:
df_features.shape

(100, 27)

In [ ]:
# Appendix

In [ ]:
def get_feature_for_runner(runner: Runner)->dict:
    previous_horse_runners = [r for r in runner.horse.runners if r.race.date < runner.race.date]
    previous_jockey_runners = [r for r in runner.jockey_entity.runners_as_jockey if r.race.date < runner.race.date]
    previous_trainer_runners = [r for r in runner.trainer_entity.runners_as_trainer if r.race.date < runner.race.date]
    
    parsed_music=parse_unibet_music(runner.race.date.year, runner.music)

    mean_place, win_ratio =None, None
    if parsed_music and len(parsed_music.events) > len(previous_horse_runners):
        mean_place =np.mean([event.rank.value for event in parsed_music.events if isinstance(event.rank.value, int) and event.rank != MusicRank.TENTH_AND_BELOW])
        win_ratio=np.mean([event.rank == MusicRank.FIRST for event in parsed_music.events])
    
    elif previous_horse_runners:
        mean_place = np.mean([int(r.position) for r in previous_horse_runners if r.position and r.position.isnumeric() and int(r.position)!=0])
        win_ratio=np.mean([r.position=='1' for r in previous_horse_runners])
        
    jockey_mean_position, jockey_win_ratio = None, None
    if previous_jockey_runners:
        jockey_mean_position = np.mean([int(r.position) for r in previous_jockey_runners if r.position and r.position.isnumeric() and int(r.position)!=0])
        jockey_win_ratio=np.mean([r.position=='1' for r in previous_jockey_runners])
    
    trainer_mean_position, trainer_win_ratio =  None, None
    if previous_trainer_runners:
        trainer_mean_position = np.mean([int(r.position) for r in previous_trainer_runners if r.position and r.position.isnumeric() and int(r.position)!=0])
        trainer_win_ratio=np.mean([r.position=='1' for r in previous_trainer_runners])
        
    return {'mean_place':mean_place, 'win_ratio':win_ratio, 'jockey_mean_position':jockey_mean_position, 'jockey_win_ratio': jockey_win_ratio,
           'trainer_mean_position':trainer_mean_position,'trainer_win_ratio':trainer_win_ratio}

In [ ]:
with create_sqlalchemy_session() as db_session:
    for r in race.runners:
        runner =db_session.query(Runner).filter(Runner.id==r.id).one()

        previous_horse_runners = [r for r in runner.horse.runners if r.race.date < runner.race.date]

        parsed_music=parse_unibet_music(runner.race.date.year, runner.music)

        if len(parsed_music.events) > len(previous_horse_runners):
            mean_place =np.mean([event.rank.value for event in parsed_music.events if isinstance(event.rank.value, int) and event.rank != MusicRank.TENTH_AND_BELOW])
            win_ratio=np.mean([event.rank == MusicRank.FIRST for event in parsed_music.events])

        else:
            mean_place = np.mean([int(r.position) for r in previous_horse_runners if r.position.isnumeric() and int(r.position)!=0])
            win_ratio=np.mean([r.position=='1' for r in previous_horse_runners])

        print({'mean_place':mean_place, 'win_ratio':win_ratio})

In [ ]:
N_HORSES_RACE =16
with create_sqlalchemy_session() as db_session:
    race_ids =db_session.execute(f"""select race_id 
from (
select r.race_id, count(1) as "n_horses"
from runners r
group by r.race_id ) as race_n_horses
where race_n_horses.n_horses={N_HORSES_RACE}""").fetchall()
    
    race_ids = [race_id[0] for race_id in race_ids]
    
    